# Corona Virus Tweets Sentiment Analysis

## Table of Content
<ul>
    <li><a href="#i">Import Packages.</a></li>
    <li><a href="#dw">Data Wrangling.</a></li>
    <li><a href="#ds">Deep Learning.</a></li>
    <li><a href="#s">Model Saving.</a></li>
</ul>

<a id="i"></a>
## Import Packages

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import re

<a id="dw"></a>
## Data Wrangling 

In [3]:
df = pd.read_csv(r"C:\Users\ameer\Downloads\Corona_NLP_train.csv/Corona_NLP_train.csv", encoding='latin-1')

df.head()

,UserName,ScreenName,Location,TweetAt,OriginalTweet,Sentiment
0,3799,48751,London,16-03-2020,@MeNyrbie @Phil_Gahan @Chrisitv https://t.co/i...,Neutral
1,3800,48752,UK,16-03-2020,advice Talk to your neighbours family to excha...,Positive
2,3801,48753,Vagabonds,16-03-2020,Coronavirus Australia: Woolworths to give elde...,Positive
3,3802,48754,NaN,16-03-2020,My food stock is not the only one which is emp...,Positive
4,3803,48755,NaN,16-03-2020,"Me, ready to go at supermarket during the #COV...",Extremely Negative


In [4]:
df = df[['OriginalTweet', 'Sentiment']]

In [5]:
df['OriginalTweet'] = df['OriginalTweet'].apply(lambda x: x.lower())
df['OriginalTweet'] = df['OriginalTweet'].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x)))

In [6]:
def remove_users(tweet):
    """Takes a string and removes retweet and @user information"""
    tweet = re.sub('(RT\s@[A-Za-z]+[A-Za-z0-9-_]+)', '', tweet)  # remove re-tweet
    tweet = re.sub('(@[A-Za-z]+[A-Za-z0-9-_]+)', '', tweet)  # remove tweeted at
    return tweet

def remove_hashtags(tweet):
    """Takes a string and removes any hash tags"""
    tweet = re.sub('(#[A-Za-z]+[A-Za-z0-9-_]+)', '', tweet)  # remove hash tags
    return tweet

def remove_av(tweet):
    """Takes a string and removes AUDIO/VIDEO tags or labels"""
    tweet = re.sub('VIDEO:', '', tweet)  # remove 'VIDEO:' from start of tweet
    tweet = re.sub('AUDIO:', '', tweet)  # remove 'AUDIO:' from start of tweet
    return tweet

In [7]:
df['OriginalTweet'] = df['OriginalTweet'].apply(remove_users)
df['OriginalTweet'] = df['OriginalTweet'].apply(remove_av)
df['OriginalTweet'] = df['OriginalTweet'].apply(remove_hashtags)

In [8]:
max_fatures = 2000
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(df['OriginalTweet'].values)
X = tokenizer.texts_to_sequences(df['OriginalTweet'].values)
X = pad_sequences(X)

In [9]:
Y = pd.get_dummies(df['Sentiment']).values
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.33, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(27575, 60) (27575, 5)
(13582, 60) (13582, 5)


<a id="dl">Deep Learning</a>

In [10]:
embed_dim = 128
lstm_out = 196

model = Sequential()
model.add(Embedding(max_fatures, embed_dim,input_length = X.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(5,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 60, 128)           256000    
                                                                 
 spatial_dropout1d (SpatialD  (None, 60, 128)          0         
 ropout1D)                                                       
                                                                 
 lstm (LSTM)                 (None, 196)               254800    
                                                                 
 dense (Dense)               (None, 5)                 985       
                                                                 
Total params: 511,785
Trainable params: 511,785
Non-trainable params: 0
_________________________________________________________________
None


In [11]:
batch_size = 32
model.fit(X_train, Y_train, epochs = 25, batch_size=batch_size)

Epoch 1/25
862/862 [==============================] - 235s 268ms/step - loss: 1.2482 - accuracy: 0.4660
Epoch 2/25
862/862 [==============================] - 234s 271ms/step - loss: 0.9412 - accuracy: 0.6464
Epoch 3/25
862/862 [==============================] - 224s 260ms/step - loss: 0.8863 - accuracy: 0.6746
Epoch 4/25
862/862 [==============================] - 222s 258ms/step - loss: 0.8488 - accuracy: 0.6884
Epoch 5/25
862/862 [==============================] - 227s 263ms/step - loss: 0.8152 - accuracy: 0.6986
Epoch 6/25
862/862 [==============================] - 237s 275ms/step - loss: 0.7817 - accuracy: 0.7132
Epoch 7/25
862/862 [==============================] - 240s 279ms/step - loss: 0.7516 - accuracy: 0.7237
Epoch 8/25
862/862 [==============================] - 238s 276ms/step - loss: 0.7246 - accuracy: 0.7360
Epoch 9/25
862/862 [==============================] - 385s 446ms/step - loss: 0.6996 - accuracy: 0.7411
Epoch 10/25
862/862 [==============================] - 220s 256m

<a id="s"></a>
## Model Saving

In [13]:
model.save("NLP_model.h5")